# Get Airflow Metadata - DAGs

# get_airflow_dag_list.py

from airflow import settings
from airflow.models import DagModel
from sqlalchemy.orm import Session

import os
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timezone, timedelta

DAG_NAME = 'get_dag_list'
dag_args= {
    'owner': 'rho715@wavve.com',
    'depends_on_past': False,
    'start_date': datetime.today() - timedelta(days=1),
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 0,
    #'on_failure_callback': alert.slack_fail_alert
}

# cron_schedule = None
cron_schedule = '55 7 * * *'
dag = DAG(DAG_NAME, catchup=False, default_args=dag_args, schedule_interval=cron_schedule, tags=['test'], is_paused_upon_creation=True)

def my_task_run():
    # Open the file for writing
    with open("/opt/airflow/data/dag_list.txt","w") as f:
        session = Session(bind=settings.engine)
        dags = session.query(DagModel).filter(DagModel.is_active == True).all()
        print("Running my task at 8am every day!")
        print(os.getcwd())

        # Write the header to the file
        f.write("DAG ID, Schedule Interval \n")

        for dag in dags:
            # Write the DAG information to the file
            f.write(f"DAG ID: {dag.dag_id}, Schedule Interval: {dag.schedule_interval}")

        session.close()
    # Close the file after the loop has completed

# Close the Session

task = PythonOperator(
    task_id="my_task",
    python_callable=my_task_run,
    dag=dag,
)

# Add the operator to the DAG using the DAG.add_task method
task